In [1]:
from pymongo import MongoClient, InsertOne, UpdateOne, UpdateMany, ReplaceOne, DeleteOne, DeleteMany

# Inisialisasi client dan koneksi ke database
client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']

print("Connected")

Connected


In [2]:
courses_collection.drop()
# Bulk insert data awal
operations = [
    InsertOne({'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}),
    InsertOne({'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}),
    InsertOne({'course': 'History 201', 'enrollments': 20, 'department': 'History'}),
    InsertOne({'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics'})
]
courses_collection.bulk_write(operations)
print('Data baru berhasil dimasukkan!\n')

# Menampilkan data awal
print("Data di dalam koleksi:")
for c in courses_collection.find():
    print(c)

# Bulk insert tambahan, course dengan beberapa enrollment
operations = [
    InsertOne({'course': 'Geography 301', 'enrollments': 15, 'department': 'Social Science'}),
    InsertOne({'course': 'Geography 301', 'enrollments': 25, 'department': 'Social Science'}),
    InsertOne({'course': 'Geography 301', 'enrollments': 20, 'department': 'Social Science'}),
    InsertOne({'course': 'Geography 301', 'enrollments': 30, 'department': 'Social Science'})
]
courses_collection.bulk_write(operations)
print('\nData baru berhasil dimasukkan!\n')

# Menampilkan data terbaru setelah penambahan
print("Data di dalam koleksi:")
for c in courses_collection.find():
    print(c)

# Menghapus data tertentu (DeleteMany)
# delete_ops = [
#     DeleteMany({'course': {'$in': ['Math 101', 'CS 102', 'History 201', 'Physics 202', 'Geography 301']}})
# ]
# courses_collection.bulk_write(delete_ops)
# print('\nData berhasi dihapus dari koleksi!\n')

# Menampilkan data setelah penghapusan
# print("Data di dalam koleksi:")
# for c in courses_collection.find():
#     print(c)

# Menghapus seluruh koleksi
# courses_collection.drop()
# print("\nKoleksi telah dihapus sepenuhnya dari database.")

Data baru berhasil dimasukkan!

Data di dalam koleksi:
{'_id': ObjectId('690f1bbe5c714823e7f0fa89'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('690f1bbe5c714823e7f0fa8a'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}
{'_id': ObjectId('690f1bbe5c714823e7f0fa8b'), 'course': 'History 201', 'enrollments': 20, 'department': 'History'}
{'_id': ObjectId('690f1bbe5c714823e7f0fa8c'), 'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics'}

Data baru berhasil dimasukkan!

Data di dalam koleksi:
{'_id': ObjectId('690f1bbe5c714823e7f0fa89'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('690f1bbe5c714823e7f0fa8a'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}
{'_id': ObjectId('690f1bbe5c714823e7f0fa8b'), 'course': 'History 201', 'enrollments': 20, 'department': 'History'}
{'_id': ObjectId('690f1bbe5c714823e7f0fa8c'), 'course': 'Physics 202', 'en

In [3]:
# Complex Filtering and Querying
print("Course dengan enrollments > 20:")
for doc in courses_collection.find({"enrollments": {"$gt": 20}}):
    print(doc)

print("\nCourse dari departemen Computer Science, Mathematics:")
query = {"$or": [{"department": "Computer Science"}, {"department": "Mathematics"}]}
for doc in courses_collection.find(query):
    print(doc)

print("\nCourse dengan enrollments > 20 dari department Social Science:")
query = {"$and": [{"enrollments": {"$gt": 20}}, {"department": "Social Science"}]}
for doc in courses_collection.find(query):
    print(doc)

Course dengan enrollments > 20:
{'_id': ObjectId('690f1bbe5c714823e7f0fa89'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('690f1bbe5c714823e7f0fa8a'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}
{'_id': ObjectId('690f1bbe5c714823e7f0fa8e'), 'course': 'Geography 301', 'enrollments': 25, 'department': 'Social Science'}
{'_id': ObjectId('690f1bbe5c714823e7f0fa90'), 'course': 'Geography 301', 'enrollments': 30, 'department': 'Social Science'}

Course dari departemen Computer Science, Mathematics:
{'_id': ObjectId('690f1bbe5c714823e7f0fa89'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics'}
{'_id': ObjectId('690f1bbe5c714823e7f0fa8a'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}

Course dengan enrollments > 20 dari department Social Science:
{'_id': ObjectId('690f1bbe5c714823e7f0fa8e'), 'course': 'Geography 301', 'enrollments': 25, 'department': 'Social Science'}
{'_id':

In [4]:
# Aggregation Framework
print("Rata-rata enrollments per department:")
pipeline = [
    {"$group": {"_id": "$department", "avg_enrollments": {"$avg": "$enrollments"}}}
]
for doc in courses_collection.aggregate(pipeline):
    print(doc)

print("\nTotal enrollments per department:")
pipeline = [
    {"$group": {"_id": "$department", "total_enrollments": {"$sum": "$enrollments"}}}
]
for doc in courses_collection.aggregate(pipeline):
    print(doc)

Rata-rata enrollments per department:
{'_id': 'History', 'avg_enrollments': 20.0}
{'_id': 'Computer Science', 'avg_enrollments': 25.0}
{'_id': 'Physics', 'avg_enrollments': 15.0}
{'_id': 'Mathematics', 'avg_enrollments': 30.0}
{'_id': 'Social Science', 'avg_enrollments': 22.5}

Total enrollments per department:
{'_id': 'History', 'total_enrollments': 20}
{'_id': 'Computer Science', 'total_enrollments': 25}
{'_id': 'Physics', 'total_enrollments': 15}
{'_id': 'Mathematics', 'total_enrollments': 30}
{'_id': 'Social Science', 'total_enrollments': 90}


In [5]:
# Data Transformation
print("\nMenampilkan hanya kolom course, department, dan enrollments:")
pipeline = [
    {"$project": {"_id": 0, "Course": "$course", "Department": "$department", "Enrollments": 1}}
]
for doc in courses_collection.aggregate(pipeline):
    print(doc)

print("\nMenambahkan kategori enrollment (High/Low):")
pipeline = [
    {"$addFields": {
        "enrollment_category": {
            "$cond": [{"$gt": ["$enrollments", 20]}, "High", "Low"]
        }
    }},
    {"$project": {"_id": 0, "course": 1, "department": 1, "enrollments": 1, "enrollment_category": 1}}
]
for doc in courses_collection.aggregate(pipeline):
    print(doc)


Menampilkan hanya kolom course, department, dan enrollments:
{'Course': 'Math 101', 'Department': 'Mathematics'}
{'Course': 'CS 102', 'Department': 'Computer Science'}
{'Course': 'History 201', 'Department': 'History'}
{'Course': 'Physics 202', 'Department': 'Physics'}
{'Course': 'Geography 301', 'Department': 'Social Science'}
{'Course': 'Geography 301', 'Department': 'Social Science'}
{'Course': 'Geography 301', 'Department': 'Social Science'}
{'Course': 'Geography 301', 'Department': 'Social Science'}

Menambahkan kategori enrollment (High/Low):
{'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematics', 'enrollment_category': 'High'}
{'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science', 'enrollment_category': 'High'}
{'course': 'History 201', 'enrollments': 20, 'department': 'History', 'enrollment_category': 'Low'}
{'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics', 'enrollment_category': 'Low'}
{'course': 'Geography 301', 'enrollme

In [6]:
# Homework 1 counting & sorting
print("Jumlah course per departemen:")
pipeline = [
    {"$group": {"_id": "$department","total_courses": {"$sum": 1}}},
    {"$sort": {"total_courses": -1}}
]

for doc in courses_collection.aggregate(pipeline):
    print(doc)

Jumlah course per departemen:
{'_id': 'Social Science', 'total_courses': 4}
{'_id': 'History', 'total_courses': 1}
{'_id': 'Computer Science', 'total_courses': 1}
{'_id': 'Physics', 'total_courses': 1}
{'_id': 'Mathematics', 'total_courses': 1}


In [7]:
# Homework 2 filtering & grouping
print("Courses dengan enrollments > 25 di departemen Computer Science:")
pipeline = [
    {"$match": {
        "department": "Computer Science",
        "enrollments": {"$gt": 25}
    }},
    {"$group": {
        "_id": "$department",
        "count_courses": {"$sum": 1},
        "average_enrollments": {"$avg": "$enrollments"},
        "total_enrollments": {"$sum": "$enrollments"}
    }}
]

for doc in courses_collection.aggregate(pipeline):
    print(doc)

Courses dengan enrollments > 25 di departemen Computer Science:


In [8]:
# Homework 3 join 2 collection

# Membuat collection baru 
students_collection = db['students']
students_collection.drop()

students_data = [
    {"student_id": 1, "name": "Alice", "course": "Math 101", "enrollment_count": 30},
    {"student_id": 2, "name": "Bob", "course": "CS 102", "enrollment_count": 25},
    {"student_id": 3, "name": "Charlie", "course": "Geography 301", "enrollment_count": 20},
    {"student_id": 4, "name": "David", "course": "Physics 202", "enrollment_count": 15},
    {"student_id": 5, "name": "Eve", "course": "History 201", "enrollment_count": 20}
]
students_collection.insert_many(students_data)
print("Data baru berhasil dimasukkan!")

# Menampilkan data mahasiswa
print("\nData Mahasiswa:")
for doc in students_collection.find():
    print(doc)

# Join courses denhan students berdasarkan nama course
pipeline = [
    {"$lookup": 
     {"from": "students",
      "localField": "course",
      "foreignField": "course",
      "as": "student_details"}},
    {"$project": {"_id": 0, "course": 1, "department": 1, "student_details.name": 1}}
]

print("\nData mahasiswa yang mengikuti course berdasarkan nama course:")
for doc in courses_collection.aggregate(pipeline):
    print(doc)

Data baru berhasil dimasukkan!

Data Mahasiswa:
{'_id': ObjectId('690f1bbf5c714823e7f0fa91'), 'student_id': 1, 'name': 'Alice', 'course': 'Math 101', 'enrollment_count': 30}
{'_id': ObjectId('690f1bbf5c714823e7f0fa92'), 'student_id': 2, 'name': 'Bob', 'course': 'CS 102', 'enrollment_count': 25}
{'_id': ObjectId('690f1bbf5c714823e7f0fa93'), 'student_id': 3, 'name': 'Charlie', 'course': 'Geography 301', 'enrollment_count': 20}
{'_id': ObjectId('690f1bbf5c714823e7f0fa94'), 'student_id': 4, 'name': 'David', 'course': 'Physics 202', 'enrollment_count': 15}
{'_id': ObjectId('690f1bbf5c714823e7f0fa95'), 'student_id': 5, 'name': 'Eve', 'course': 'History 201', 'enrollment_count': 20}

Data mahasiswa yang mengikuti course berdasarkan nama course:
{'course': 'Math 101', 'department': 'Mathematics', 'student_details': [{'name': 'Alice'}]}
{'course': 'CS 102', 'department': 'Computer Science', 'student_details': [{'name': 'Bob'}]}
{'course': 'History 201', 'department': 'History', 'student_details